In [3]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt

# 1. Recopilación de fuentes y categorización

In [1]:
# #rutas (ALEJANDO - PERSONAL)

# # Ruta base donde se encuentran los archivos shapefile
# base_path = r"C:\Users\Alejandro\Downloads\DATA\SHP"

# output_path_excel = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_2020.xlsx"
# output_path_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

#rutas (ALEJANDO - SSD)

# Ruta base donde se encuentran los archivos shapefile
base_path = r"Z:\VOCES\DATA\SHP"

output_path_excel = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_2020.xlsx"
output_path_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

In [4]:
#1.2 Carga de capas de información puntual, de ambos años, en Jupyter

# Lista para almacenar los GeoDataFrames de cada estado
gdfs = []

# Recorrer los números de estado (del 01 al 32)
for i in range(1, 33):
    # Formatear el número del estado como 'XX' (ejemplo: 01, 02, 03...32)
    state_code = f"{i:02d}"
    # Crear la ruta completa del archivo shapefile
    file_path = os.path.join(base_path, f"{state_code}sip.shp")
    
    # Cargar el shapefile y añadirlo a la lista
    gdf_state = gpd.read_file(file_path)
    gdfs.append(gdf_state)

# Unir todos los GeoDataFrames en uno solo
gdf_2020 = gpd.pd.concat(gdfs, ignore_index=True)

# Verificar la unión exitosa
print(f"Total de registros en el archivo unido: {len(gdf_2020)}")

#guardar en gpkg
gdf_2020.to_file(output_path_gpkg, layer='Eq_full_2020')



#1.3 Desplegado de campos únicos

# Agrupar por 'GEOGRÁFICO' y obtener los valores únicos de 'TIPO' para cada uno
diccionario_geografico_tipo = {}

# Recorrer cada valor único de GEOGRAFICO
for geografico in gdf_2020['GEOGRAFICO'].unique():
    # Filtrar los datos correspondientes al valor de 'GEOGRAFICO' actual
    subset = gdf_2020[gdf_2020['GEOGRAFICO'] == geografico]
    
    # Obtener los valores únicos de 'TIPO' asociados a este 'GEOGRAFICO'
    tipos = subset['TIPO'].unique()
    
    # Añadir al diccionario, si no tiene subniveles, ponemos una lista vacía
    diccionario_geografico_tipo[geografico] = list(tipos) if len(tipos) > 0 else []

# Mostrar el diccionario creado
for geo, tipos in diccionario_geografico_tipo.items():
    print(f"{geo}: {tipos}")


#1.4 Desplegado de valores únicos en campos X e Y
#1.5 Exportación de ambas listas en .xlsx

# Convertir el diccionario en un DataFrame para exportar a Excel
data = []

# Recorrer el diccionario y estructurarlo en filas
for geografico, tipos in diccionario_geografico_tipo.items():
    if tipos:  # Si hay subniveles en 'TIPO'
        for tipo in tipos:
            data.append([geografico, tipo])
    else:
        data.append([geografico, None])  # Si no hay subniveles, ponemos 'None' en 'TIPO'

# Crear DataFrame
df_diccionario = pd.DataFrame(data, columns=['GEOGRAFICO', 'TIPO'])

# Exportar a archivo Excel
df_diccionario.to_excel(output_path_excel, index=False)

print(f"Diccionario exportado exitosamente a {output_path_excel}")


Total de registros en el archivo unido: 440164
Centro de Asistencia Médica: ['Centro de Salud', 'Hospital', 'Centro de Rehabilitación']
Escuela: ['Primaria', 'Preescolar', 'Secundaria', 'Medio Superior', 'Superior', 'Mixto']
Templo: ['No Aplica']
Cementerio: ['No Aplica']
Pozo: ['Agua', 'Petróleo', 'Gas']
Instalación Deportiva o Recreativa: ['Cancha', 'Parque', 'Unidad Deportiva', 'Jardín', 'Lienzo Charro', 'Áreas Verdes', 'Estadio', 'Plaza de Toros', 'Campo de Golf', 'Velódromo', 'Balneario', 'Hipódromo', 'Galgódromo', 'Zoológico', 'Alberca Olímpica', 'Campo de Tiro', 'Autódromo', 'Acuario']
Tanque de Agua: ['Tanque Elevado', 'Caja de Agua']
Instalación Gubernamental: ['Palacio de Gobierno', 'Palacio Municipal']
Mercado: ['No Aplica']
Plaza: ['No Aplica']
Instalación Diversa: ['Monumento u Obelisco', 'Silo', 'Planta de Tratamiento de Agua', 'Gasolinera', 'Museo', 'Centro de Investigación', 'Teatro', 'Edificación Cultural', 'Centro de Espectáculos', 'Cine', 'Estación de Gas', 'Observat

# 2. Ponderación y niveles de servicio

In [5]:
# #rutas (ALEJANDRO - PERSONAL)
# equipamientos_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"
# Diccionario_comentado = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_pond_2020.xlsx"

#rutas (ALEJANDRO - SSD)
equipamientos_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"
Diccionario_comentado = r"Z:\VOCES\DATA\\Tratados\NAC\EQUIPAMIENTOS\Diccionario_pond_2020.xlsx"

In [6]:
# Cargar el diccionario actualizado con las nuevas columnas
diccionario = pd.read_excel(Diccionario_comentado)

# Unir el diccionario con el geodataframe gdf_2020 (asegurarse de que haya un campo en común, por ejemplo 'GEOGRAFICO' y 'TIPO')
gdf_2020 = gdf_2020.merge(diccionario, on=['GEOGRAFICO', 'TIPO'], how='left')

# Filtrar por categorías y exportar a archivos individuales

# Crear una lista de categorías
categorias = ['Educación', 'Salud', 'Recreación', 'Servicios']

# Crear y exportar un GeoDataFrame por cada categoría a una capa dentro del geopackage
for categoria in categorias:
    # Filtrar los equipamientos de la categoría actual
    gdf_categoria = gdf_2020[gdf_2020['CATEGORIA'] == categoria]
    
    # Exportar la categoría como una nueva capa en el geopackage
    gdf_categoria.to_file(equipamientos_gpkg, layer=categoria, driver="GPKG")
    
    print(f"Exportado: {categoria} a la capa {categoria} en el geopackage")

Exportado: Educación a la capa Educación en el geopackage
Exportado: Salud a la capa Salud en el geopackage
Exportado: Recreación a la capa Recreación en el geopackage
Exportado: Servicios a la capa Servicios en el geopackage


# 3. Trabajo a nivel manzana

In [ ]:
# #rutas (ALEJANDRO - PERSONAL)
manzanas_gpkg_path = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\MZ\Manzanas_NAC.gpkg"
equipamientos_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

#rutas (ALEJANDRO - SSD)
manzanas_gpkg_path = r"Z:\"
equipamientos_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

In [ ]:
#3.1: Cargar el archivo de manzanas

# Cargar el archivo de manzanas
gdf_manzanas = gpd.read_file(manzanas_gpkg_path, layer='nombre_de_la_capa_de_manzanas')

# Mostrar algunas filas para confirmar la carga
gdf_manzanas.head()


In [ ]:
#leer gpkg equipamientos

gdf_2020.read_file(equipamientos_gpkg, layer='Eq_full_2020')

In [ ]:
#3.2: Conteo de equipamientos por manzana (con buffer de 10m)

# Crear un buffer de 10 metros alrededor de las manzanas
gdf_manzanas['buffer_10m'] = gdf_manzanas.geometry.buffer(10)

# Inicializar las columnas para contar equipamientos en cada categoría
gdf_manzanas['num_edu'] = 0
gdf_manzanas['num_sal'] = 0
gdf_manzanas['num_rec'] = 0
gdf_manzanas['num_ser'] = 0

# Filtrar equipamientos por categoría y contar cuántos caen dentro de cada buffer
for categoria in categorias:
    gdf_categoria = gdf_2020[gdf_2020['CATEGORIA'] == categoria]
    
    # Hacer un join espacial entre los buffers y los equipamientos
    joined = gpd.sjoin(gdf_manzanas[['buffer_10m']], gdf_categoria, how='left', op='contains')

    # Contar equipamientos por manzana
    if categoria == 'Educación':
        gdf_manzanas['num_edu'] = joined.groupby(joined.index)['index_right'].count()
    elif categoria == 'Salud':
        gdf_manzanas['num_sal'] = joined.groupby(joined.index)['index_right'].count()
    elif categoria == 'Recreación':
        gdf_manzanas['num_rec'] = joined.groupby(joined.index)['index_right'].count()
    elif categoria == 'Servicios':
        gdf_manzanas['num_ser'] = joined.groupby(joined.index)['index_right'].count()

# Revisar el resultado
gdf_manzanas.head()


In [ ]:
#3.3: Aplicación de radios de influencia para los equipamientos

# Definir los valores de radio en metros para cada nivel de servicio
radio_niveles = {1: 1000, 2: 5000, 3: 12000}  # 1 km, 5 km, 12 km

# Crear los buffers basados en el nivel de servicio (campo 'RADIO')
gdf_2020['buffer_influencia'] = gdf_2020.apply(lambda row: row.geometry.buffer(radio_niveles[row['RADIO']]), axis=1)

# Revisar el resultado
gdf_2020.head()


In [ ]:
# 3.4: Sumar los valores de ponderación por manzana

# Inicializar las columnas de importancia por categoría
gdf_manzanas['IMP_EDU'] = 0
gdf_manzanas['IMP_SAL'] = 0
gdf_manzanas['IMP_REC'] = 0
gdf_manzanas['IMP_SER'] = 0

# Realizar el join espacial entre los buffers de equipamientos y las manzanas
for categoria in categorias:
    gdf_categoria = gdf_2020[gdf_2020['CATEGORIA'] == categoria]

    # Join espacial entre manzanas y equipamientos por categoría
    joined = gpd.sjoin(gdf_manzanas, gdf_categoria[['buffer_influencia', 'IMPORTANCIA']], how='left', op='intersects')
    
    # Sumar la importancia de los equipamientos que caen dentro de cada manzana
    if categoria == 'Educación':
        gdf_manzanas['IMP_EDU'] = joined.groupby(joined.index)['IMPORTANCIA'].sum()
    elif categoria == 'Salud':
        gdf_manzanas['IMP_SAL'] = joined.groupby(joined.index)['IMPORTANCIA'].sum()
    elif categoria == 'Recreación':
        gdf_manzanas['IMP_REC'] = joined.groupby(joined.index)['IMPORTANCIA'].sum()
    elif categoria == 'Servicios':
        gdf_manzanas['IMP_SER'] = joined.groupby(joined.index)['IMPORTANCIA'].sum()

# Revisar el resultado
gdf_manzanas.head()


In [ ]:
# 3.5: Visualización estadística de los valores de importancia por categoría

# Resumen estadístico de las columnas de importancia
print(gdf_manzanas[['IMP_EDU', 'IMP_SAL', 'IMP_REC', 'IMP_SER']].describe())

# Visualización gráfica de los valores de importancia por categoría
gdf_manzanas[['IMP_EDU', 'IMP_SAL', 'IMP_REC', 'IMP_SER']].plot(kind='box', title='Importancia por Categoría')
plt.show()
